In [35]:
import os, sys
import transformers
import torch
import torch.nn as nn

from transformers import pipeline
from DGXutils import GetLowestGPU, GetFileNames

sys.path.append('../')

from utils import preprocess as pp
from utils.LLM import GraphLLM
from utils.multiplex import Multiplex
from utils.textualize import *
from utils.bio_data_generator import *

# Step 1: Load Networks as `NX` Objects

In [39]:
# load in graphs
graph_path = '../data/DREAM4_gold_standards/'
graph_names = GetFileNames(graph_path, ".tsv")
print(graph_names)

['insilico_size10_1_goldstandard.tsv', 'insilico_size10_2_goldstandard.tsv', 'insilico_size10_3_goldstandard.tsv', 'insilico_size10_4_goldstandard.tsv', 'insilico_size10_5_goldstandard.tsv']


In [42]:
# convert to networkx objects
nx_graphs = []
for graph in graph_names:
    nx_graphs.append(pp.convert_to_nx(graph_path + graph))
print("Done!")

Done!


# Step 1(alt): Load Networks as `Multiplex` object

In [5]:
flist_name = '../data/DREAM4_gold_standards/flist.tsv'
mp = Multiplex(flist_name)

# Step 2: Textualize Graphs (Ken's Code)

In [6]:
textualize = load_textualizer['edges']
grap_text = textualize(mp)

# Step 3: Make Dataloader
* dataloader returns dict with keys `["desc"]`,`["question"]`,`["label"]`

In [2]:

flist_name = '../data/DREAM4_gold_standards/mono_flist.tsv'
mp = Multiplex(flist_name)

In [3]:
outpath = '../data/DREAM4_gold_standards'
generate_data_mono('edges', outpath, flist_name, 30)

# train samples:  54
# val samples:  18
# test samples:  18


# Step 4: Load In Encoder + LLM

In [ ]:
model = GraphLLM() # need to add args

# Step 5: Perform Initial Untrained Inference

# Step 6: Train Model

# Step 7: Evaluate After Training